Importy

In [1]:
import re, os
from collections import Counter
import numpy as np
from numpy import linalg as LA
from heapq import nlargest

Wczytanie tekstu - 1144 artykułów z wikipedii

In [2]:
documents = [[] for i in range(1144)]
bag_of_words = set()

i = 0
for filename in os.listdir('text'):
    with open(os.path.join('text', filename)) as file:
        for line in file:
            wordList = list(map(lambda x: x.lower(), line.split()))
            documents[i] += wordList
            bag_of_words |= set(wordList)
    i += 1
len(documents)

1144

Częstości słów

In [3]:
word_frequency = []

for doc in documents:
    words_dict = dict.fromkeys(bag_of_words,0)
    for word in doc:
        words_dict.update({word:words_dict.get(word)+1})
    word_frequency.append(words_dict)

Słownik wystąpień słów w dokumentach

In [4]:
def get_term_by_document_matrix():
    words_num = len(bag_of_words)
    term_by_document_matrix = np.zeros((words_num,1144))
    words_list = list(bag_of_words)

    for i in range(1144):
        for index,word in enumerate(words_list):
            term_by_document_matrix[index,i] = word_frequency[i].get(word)
            
    return term_by_document_matrix
            
term_by_document_matrix = get_term_by_document_matrix()

Inverse Document Frequency Vector

In [5]:
idf_vector = []

N = 1144
for index,word in enumerate(list(bag_of_words)):
    words_number = 0
    for i in range(N):
        if(term_by_document_matrix[index,i] > 0):
            words_number += 1
    if(words_number != 0):
        idf_vector.append(np.log10(N/words_number))


Miara podobieństwa frazy z dokumentami

In [6]:
def similarity_metric(sentence, veclen):
    query = map(lambda x: x.lower(), sentence.split())
    
    query_bag_of_words = dict.fromkeys(bag_of_words,0)
    for word in query:
        if(word in bag_of_words):
            query_bag_of_words.update({word:query_bag_of_words.get(word)+1})
    
    q = []
    for index,word in enumerate(list(bag_of_words)):
        q.append(query_bag_of_words.get(word))
    
    similarity_rate = {}
    for i in range(1144):
        dj = term_by_document_matrix[:,[i]]
        q_norm = LA.norm(q)
        dj_norm = LA.norm(dj)
        cosj = np.dot(q,dj)/(q_norm*dj_norm)
        similarity_rate.update({i:cosj})
        
    return nlargest(veclen, similarity_rate, key=similarity_rate.get)

Teraz sprawdźmy jak to działa

In [7]:
sentence = "politician cat man"
similarity_metric(sentence,20)

[917,
 355,
 850,
 688,
 731,
 781,
 198,
 553,
 311,
 55,
 654,
 789,
 872,
 1018,
 572,
 777,
 706,
 1120,
 10,
 226]

Wynik po odszumianiu przez SVD i low rank aproximation

In [8]:
U, s, V = np.linalg.svd(term_by_document_matrix, full_matrices=False)
S = np.diag(s)
term_by_document_matrix = np.dot(U, np.dot(S, V))

similarity_metric(sentence,20)

[917,
 355,
 850,
 688,
 731,
 781,
 198,
 553,
 311,
 55,
 654,
 789,
 872,
 1018,
 572,
 777,
 706,
 1120,
 10,
 226]

Jak widać odszumianie nie zmieniło wynikiów.

Usuwamy efekty SVD, przystępujemy do testów zastosowania Inveres Document Frequency

Mnożymy wystąpienia słów przez idf, ponownie wyszukujemy dokumentów podobnych

In [10]:
term_by_document_matrix = get_term_by_document_matrix()
for i in range(1000):
    for j in range(len(idf_vector)):
        term_by_document_matrix[j,i] *= idf_vector[j]
      
after_IDF = similarity_metric(sentence,20)

Jak widać różnica przed i po zastosowaniu IDF jest znacząca:

In [11]:
print(after_IDF)

[917, 850, 731, 553, 972, 10, 355, 960, 478, 766, 69, 262, 448, 545, 688, 450, 885, 883, 45, 894]


Na koniec test SVG i IDF razem:

In [12]:
U, s, V = np.linalg.svd(term_by_document_matrix, full_matrices=False)
S = np.diag(s)
term_by_document_matrix = np.dot(U, np.dot(S, V))

similarity_metric(sentence,20)

[917,
 850,
 731,
 553,
 972,
 10,
 355,
 960,
 478,
 766,
 69,
 262,
 448,
 545,
 688,
 450,
 885,
 883,
 45,
 894]

Jak widać, IDF znacząco wpłynęło na wyniki, SVD przed i po IDF nie miało większego wpływu na wyniki w tym przypadku.